In [ ]:
import sys
from pathlib import Path
import pandas as pd
from lc120_noise import common
from dataclasses import dataclass, asdict
import numpy as np
from typing import List

In [ ]:
def calc_rms_noise(voltages, dt):
    # Remove DC offset (mean subtraction)
    ac_signal = voltages - np.mean(voltages)
    
    # Integrate power = ∫ v(t)^2 dt
    noise_energy = np.sum(ac_signal**2) * dt
    
    # RMS noise = sqrt(energy / total time)
    rms_noise = np.sqrt(noise_energy / (dt * len(voltages)))
    return rms_noise

In [ ]:
@dataclass
class RunLine:
    trace: List[float] # average the traces to a single trace
    current: float
    temp: float
    rms_noise: float

fstructure = common.FileStructure(Path("./data/demo"))
runs = fstructure.get_all_runs()
for run in runs:
    traces = run.load_scope_traces()
d = []
    
for run in runs:
    trace, header = run.load_scope_traces()[0]
    dt = float(header[":WAV:XINC?"])
    line = RunLine(trace=trace, temp=run.temp, current=run.current, 
                   rms_noise=calc_rms_noise(trace, dt))
    d.append(asdict(line))
df = pd.DataFrame(d)


In [ ]:
import matplotlib.pyplot as plt

pivot = df.pivot(index='temp', columns='current', values='rms_noise')
plt.figure(figsize=(8, 6))

# Use imshow or pcolormesh
plt.pcolormesh(pivot.columns, pivot.index, pivot.values, shading='auto', cmap='viridis')
plt.colorbar(label='RMS Voltage (V)')
plt.xlabel('Current (mA)')
plt.ylabel('Temperature (°C)')
plt.title('RMS Voltage vs. Current and Temperature')
plt.tight_layout()
plt.show()